In [15]:
pip install pattern3

In [16]:

#For training this classifier we need pattern, nltk (including the corpus), re and csv modules#

from pattern3.vector import Model, Document, BINARY, SVM, kfoldcv, IG, SLP,KNN, NB
from pattern3.db import csv
#from pattern3.en import ngrams
from pattern3.vector import stem, PORTER, LEMMA
from nltk.corpus import stopwords
import re
import csv as csv1


In [17]:

data = csv('Facebook_dataset.csv')
data = [[message, int(side_effect_indicator)] for message, side_effect_indicator in data]

#List of nltk stopwords
stop = [u'i', 'm', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself',
             u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its',
             u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom',
             u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being',
             u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but',
             u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against',
             u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up',
             u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when',
             u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor',
             u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don',
             u'should', u'now']


In [22]:
# Adding medicine names and obvious names into the stop words
#
# The following list was extacted essentially from wikipedia, webmd and mayoclinic websites. 
# These are all related to Glucophage and they conceptually mean the same thing and hence it should be removed
# We want data only related to adverse drug reactions  
# Pioglitazone is marketed as trademarks Actos in the USA, Canada, the UK and Germany, Glustin in Europe, 
# Glizone and Pioz in India by Zydus Cadila and USV respectively and Zactos in Mexico by Takeda Pharmaceuticals.
similar_mean_list1 = ["diabetes","actos", "pioglitazone hydrochloride", "pioglitazone",  "glustin", "glizone", "pioz", "zactos"]
#The name "Metformin" is the BAN(British Approved Name), USAN(United States Approved Name ) and INN(International Nonproprietary Names) 
#for the medication. It is sold under several trade names, including Glucophage XR, Carbophage SR, Riomet, 
#Fortamet, Glumetza, Obimet, Gluformin, Dianben, Diabex, Diaformin, Siofor, Metfogamma and Glifor.
similar_mean_list2 = ["medformin","glucophage", "metformin", "glucophage xr", "metformin hydrochloride", "carbophage sr", "riomet", "fortamet", "glumetza", "obimet", "gluformin", "dianben", "diabex", "diaformin", "siofor","metfogamma", "riomet"]
#Byetta and Bydureon are injectable medications used to treat type 2 diabetes.
similar_mean_list3 = ["byetta", "bydureon", "exenatide"]
#Liraglutide is marketed under the brand name Victoza in U.K. UAE, Kuwait, India and Saxenda in Australia,
#Iran, Israel, Canada, Brazil, and the U.S
similar_mean_list4 = ["victoza", "liraglutide", "saxenda"]
#Canagliflozin, sold under the brand name Invokana among others, 
#is a medication used to treat type 2 diabetes.
similar_mean_list5 = ["invokana", "canagliflozin"]
#Rosiglitazone (trade name Avandia) is an antidiabetic drug in the thiazolidinedione class. 
similar_mean_list6 = ["avandia", "rosiglitazone"]
# these are the types of insulin which mean the same thing
similar_mean_list7 = ["insulin glargine",  "lantus", "toujeo", "abasaglar", "basaglar","insulin"]
#Januvia (sitagliptin) and (sitagliptin/metformin) are oral diabetes medicines
similar_mean_list8 = ["sitagliptin", "janumet", "januvia", "juvisync"]
#Glimepiride, sold under the trade name Amaryl among others, is a medication used to treat diabetes mellitus type 2.
similar_mean_list9 = ["amaryl", "glimepiride", "gleam", "k-glim-1", "glucoryl",  "glimpid", "glimy"]
# remove words related to diabetes
similar_mean_list = similar_mean_list1 + similar_mean_list2 + similar_mean_list3 + similar_mean_list4 + similar_mean_list5 + similar_mean_list6 + similar_mean_list7 + similar_mean_list8 + similar_mean_list9
# Removing these stop words and general cleaning
newsimilar_mean_list = []

#converting to unicode#
for r in similar_mean_list:
    newsimilar_mean_list.append(r.encode("utf-8"))
    
#now stop words include medicine names and common words in English#
stop = stop + newsimilar_mean_list

#This will be our dataset, extracted the non-null and non-empty comments 
refineddata = [[c[0].lower(),c[1]] for c in data if c[0] != "" and c[0] != None]


In [51]:
#Function to convert the text into features(single words, bigrams and trigrams)#
def features(message):
    # Removing these stop words and general cleaning
    singlegrams =  [i for i in message.split() if i not in stop] 
    singlegramsrefined = []
    #Stemming the single words
    #The stemming takes to unusual roots, so lemmatization is applied
    for k in singlegrams:
        r = stem(k, stemmer=LEMMA)
        singlegramsrefined.append(r)
    newmessage = " ".join(singlegramsrefined)
    # Removing numbers
    newmessage = re.sub("[^A-Za-z]", " ", newmessage)
    # Removing stopwords
    newmessage = re.sub(r'[^\w]', ' ', newmessage)
    #Again splitting to single grams
    singlegrams= [i for i in newmessage.split()]
    singlegramsrefined2 = []

    for word in singlegrams:
        singlegramsrefined2.append(word)
    
    totalgrams = singlegramsrefined2     
    return totalgrams

In [52]:
#The cleaned input dataset is passed to features function 
refineddata1 = [(features(c[0]),c[1]) for c in refineddata]
#print(refineddata1)
#Each datapoint becomes a pattern document here; The type represents the label for each document#
#A Document is an unordered bag-of-words representation of a given string, dictionary of (word, count)-items, Sentence or Text.
refineddata2 = [Document(message, type=sideeffectindicator) for message, sideeffectindicator in refineddata1]
#print(refineddata2)
# Defining the model using the documents; feature weight is Information gain; 
# The weight parameter can be set to TF (relative term frequency), 
# TFIDF, (term frequency vs. document frequency), IG (information gain), BINARY (0 or 1) or None (original weights).
# Bag-of-words means that the word order in the given text is discarded. 
# Instead, words are counted using the words(), count() and stem() and functions.
model = Model(documents=refineddata2, weight=IG)

#Top 50 features selected#
features=model.feature_selection(top=50)

#If medicine names are present in the features, they are removed #
refinedfeatures = []
for i in features:
    if i not in similar_mean_list:
        refinedfeatures.append(i)

#After few rounds of examining the top features the following top features will be removed to make the model more general#
        
skipList1 = [u'mg',u'release',u'slow', u'release',u'take', u'metformin',u'mg', u'twice',u'extended', u'release']

skipList2 = [u'glipizide',u'metformin', u'i', u'day', u'twice', u'started',u'switched',u'months',u'stopped']

skipList3 = [u'made',u'med', u'insulin',u'morning']

skipList4 = [u'taking',u'gliclazide',u'acid', u'changing']

skipList5 = [u'form',u'first', u'started',u'fog', u'storm',u'storm', u'low']

skipList6 = [ u'tested',u'like',u'storm',u'lipoic', u'acid', u'mg']

skipList7 = [ u'horrible',u'metformin', u'made', u'sick',u'problem',u'one']

skipList8 = [u'told',u'took']

skipList9 = [u'x',u'tablet',u'glyberide',u'jentadueto',u'snd', u'time',u'time']

skipList10 = [u'glyburide', u'glyburide',u'im']

skipList11 = [u'm',u'doctor',u'started',u'glimeperide']

skipList12 = [u'levimir',u'lisinopril',u'metformine']

skipList = skipList1 + skipList2 + skipList3 + skipList4

skipList = skipList + skipList5 + skipList6 + skipList7 + skipList8 + skipList9 + skipList10 + skipList11 + skipList12

#Count number of features in the avoid list
print(len(skipList))

#count number of features that were there before skip list
print(len(refinedfeatures))

cleanFeatures = [i for i in refinedfeatures if i not in skipList]

#count number of features reamining after filtering out avoid list
print(cleanFeatures)

#model redefines
model = model.filter(features=cleanFeatures)

#This will give k fold cross validation results; We can also try SVM, SLP, KNN
kfoldcv(KNN, model)

#Writing the features and their weights to a csv file
listofFeatures = []

for i in cleanFeatures:
    innerList= [i,model.ig(i)]
    listofFeatures.append(innerList)

f = open("FeaturesKNN.csv", "w")
writer = csv1.writer(f,delimiter=',')

for item in listofFeatures:
    #print(item)
    writer.writerow(item)
f.close()
print("Features written to csv file")

65
46
['bathroom', 'cramps', 'diarrhea', 'effects', 'gas', 'nausea', 'pancreatitis', 'stomach', 'tummy', 'upset', 'vomiting', 'allergic', 'nauseated', 'caused', 'pounds', 'nauseous', 'gut', 'weeks', 'hair', 'gi', 'empty', 'effect', 'currently', 'abdominal', 'bowels', 'diarrhoea', 'rash', 'lost', 'awful', 'dose', 'cramping', 'gained']
Features written to csv file
